In [1]:
from torchsummary import summary
from python_lib.modules import *
import torch
import numpy as np
import os

c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html


In [26]:
model = ECAPA_TDNN(input_size = 2, channels=[8,8,8,8,16], lin_neurons=6, device = "cuda").to("cuda")
input_feats = torch.rand([10,2,64]).to("cuda")
output = model(input_feats)
output.shape

torch.Size([10, 6])

# Entire ECAPA-TDNN Model overview

In [27]:
# Default Activation fn = Relu and group = 1

initialblock = TDNNBlock(in_channels=2,
                        out_channels=8,
                        kernel_size=5,
                        dilation= 1).to("cuda")

x0 = initialblock(input_feats)
print("Initial out: ",x0.shape)

seres2_1 = SERes2NetBlock(in_channels=8,
                          out_channels=8, 
                          res2net_scale=8, 
                          se_channels=128, 
                          kernel_size=3, 
                          dilation=2).to("cuda")

x1 = seres2_1(x0)
print("SERes2_1 out: ",x1.shape)

seres2_2 = SERes2NetBlock(in_channels=8,
                          out_channels=8, 
                          res2net_scale=8, 
                          se_channels=128, 
                          kernel_size=3, 
                          dilation=3).to("cuda")

x2 = seres2_2(x1)
print("SERes2_2 out: ",x2.shape)

seres2_3 = SERes2NetBlock(in_channels=8,
                          out_channels=8, 
                          res2net_scale=8, 
                          se_channels=128, 
                          kernel_size=3, 
                          dilation=4).to("cuda")

x3 = seres2_3(x2)
print("SERes2_3 out: ",x3.shape)

mfa = TDNNBlock(in_channels=(8 * 3),
                out_channels=16,
                kernel_size=1,
                dilation= 1).to("cuda")

x4 = mfa(torch.cat([x1,x2,x3], dim=1))
print("mfa out: ",x4.shape)

asp = AttentiveStatisticsPooling(16, 128, True).to("cuda")
x5 = asp(x4)
x6 = nn.BatchNorm1d(16 * 2).to("cuda")(x5)
print("asp out: ",x6.shape)

# Remember to x2 to account for the mean & std in asp
final = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=16 * 2, out_features=6),
        ).to("cuda")

y = final(x6)
print("final out: ",y.shape)

Initial out:  torch.Size([10, 8, 64])
SERes2_1 out:  torch.Size([10, 8, 64])
SERes2_2 out:  torch.Size([10, 8, 64])
SERes2_3 out:  torch.Size([10, 8, 64])
mfa out:  torch.Size([10, 16, 64])
asp out:  torch.Size([10, 32, 1])
final out:  torch.Size([10, 6])


Parameter containing:
tensor([[ 1.4033e-01, -1.2670e-01,  1.7172e-01,  5.4900e-02,  1.0922e-01,
          9.3102e-02,  7.8937e-02, -1.1522e-01,  4.2454e-02,  4.5778e-03,
          5.0761e-02,  1.0475e-01,  1.4638e-01,  2.4510e-02,  3.5282e-02,
         -5.4543e-02,  5.0777e-02,  1.1076e-02, -1.3098e-01, -4.9563e-02,
         -1.3543e-01, -1.0652e-01, -5.3257e-02, -1.7073e-01,  1.6139e-01,
         -1.1455e-01, -8.2617e-02,  1.4518e-01,  5.0413e-02, -3.8235e-03,
          9.2700e-05,  1.2762e-02],
        [ 1.1888e-01, -1.3906e-03, -2.0703e-02, -2.1914e-02, -3.9449e-02,
         -2.1847e-02,  1.1271e-01,  6.3061e-02,  5.7661e-02,  1.0696e-02,
         -6.0790e-02, -1.5986e-01, -1.6143e-01, -9.8007e-03, -9.7754e-02,
         -1.4597e-01, -3.4645e-02, -7.8234e-03, -1.5442e-01, -2.6707e-02,
         -1.4591e-01, -1.9762e-02, -1.3433e-01, -1.0067e-01,  1.0565e-01,
          3.1718e-02,  4.8090e-03,  1.1344e-01,  1.1498e-01, -1.3357e-02,
          1.0456e-01, -7.8233e-02],
        [ 1.3027e-

## Disecting SERes2NetBlock

Since in_channel == out_channel, shortcut is not necessary

In [4]:
tdnn1 = TDNNBlock(in_channels=8, 
                  out_channels=8, 
                  kernel_size=1, 
                  dilation=1).to("cuda")

z0 = tdnn1(x0)
print(z0.shape)
res2net = Res2NetBlock(in_channels=8, 
                       out_channels=8, 
                       scale=8, 
                       kernel_size=3, 
                       dilation=1).to("cuda")

z1 = res2net(z0)
print(z1.shape)

tdnn2 = TDNNBlock(in_channels=8, 
                  out_channels=8, 
                  kernel_size=1, 
                  dilation=1).to("cuda")

z2 = tdnn2(z1)
print(z2.shape)

se_block = SEBlock(in_channels=8, 
                   se_channels=128, 
                   out_channels=8).to("cuda")

z3 = se_block(z2)
print((z3 + x0).shape)


torch.Size([10, 8, 64])
torch.Size([10, 8, 64])
torch.Size([10, 8, 64])
torch.Size([10, 8, 64])


## Dissecting Res2NetBlock

Here Scale = 8

In [5]:
scale = 8
in_channel = 8//scale # 1
hidden_channel = 8//scale # 1

block1 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block2 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block3 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block4 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block5 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block6 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block7 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")

chunks = torch.chunk(z0, 8, 1)
# for i in chunks:
#     print(i.shape)

b7 = block7(chunks[0])
print(chunks[0].shape)
print(b7.shape)

torch.Size([10, 1, 64])
torch.Size([10, 1, 64])


In [6]:
from speechbrain.nnet.CNN import Conv1d as _Conv1d

class Conv1d(_Conv1d):
    """1D convolution. Skip transpose is used to improve efficiency."""

    def __init__(self, *args, **kwargs):
        super().__init__(skip_transpose=True, *args, **kwargs)

c1 = Conv1d(in_channels=1,
            out_channels=1,
            kernel_size=3,
            dilation=1,
            groups=1).to("cuda")

c2 = nn.Conv1d(1,1,3,1,0,1).to("cuda")

print(c1(chunks[0]).shape)
print(c2(chunks[0]).shape)

torch.Size([10, 1, 64])
torch.Size([10, 1, 62])


In [7]:
c1.conv.weight

Parameter containing:
tensor([[[-0.2839,  0.4532,  0.0050]]], device='cuda:0', requires_grad=True)

In [112]:
input_feats = torch.rand([1,2,64]).to("cuda")
input_feats_np = input_feats.cpu().detach().numpy()
input_feats_np = np.reshape(input_feats_np, (2,64))
print(input_feats_np)
dim = input_feats_np.shape

flatten_inputs = input_feats_np.flatten()
with open(os.path.join("ECAPAweights", f"inputseres2_2.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_inputs.tobytes())

[[3.4579635e-03 8.5556924e-02 1.6787881e-01 5.9979117e-01 7.0609242e-01
  2.0156753e-01 3.1571430e-01 4.6073532e-01 1.6631895e-01 6.2462986e-02
  6.8170667e-02 8.8588482e-01 8.7375546e-01 7.9144663e-01 1.6559595e-01
  8.1799096e-01 9.0489054e-01 7.4740523e-01 8.4028590e-01 1.6255111e-01
  9.8915315e-01 8.0721277e-01 8.1081057e-01 1.3551807e-01 3.8094282e-02
  3.1013268e-01 6.4457613e-01 8.7095791e-01 4.8005295e-01 1.8205297e-01
  4.7799712e-01 6.2338644e-01 6.7343855e-01 3.7089330e-01 6.1123955e-01
  1.5339512e-01 2.6910949e-01 7.6813310e-01 9.4713753e-01 4.9810529e-01
  6.8790269e-01 4.8629427e-01 8.7077218e-01 7.3044378e-01 3.5490030e-01
  6.8805087e-01 1.7553443e-01 1.7682391e-01 9.0975273e-01 5.5467367e-01
  8.3731526e-01 3.2127082e-01 7.5149983e-01 3.1764024e-01 2.2891271e-01
  3.0719811e-01 9.0164089e-01 1.4639950e-01 9.5907748e-02 8.7761879e-04
  8.9419293e-01 4.7473061e-01 3.4011954e-01 8.2355344e-01]
 [8.8763207e-01 7.0542216e-01 9.7182018e-01 2.3045540e-03 4.4270915e-01
  2.6

In [9]:
import torch.nn.functional as F
input1 = F.pad(input_feats, (2,0))
input1

tensor([[[0.0000, 0.0000, 0.5059, 0.7251, 0.0422, 0.2820, 0.7079, 0.3893,
          0.4327, 0.0163, 0.9783, 0.5213, 0.3836, 0.2455, 0.3622, 0.9073,
          0.1888, 0.7879, 0.5953, 0.0281, 0.5460, 0.7562, 0.2448, 0.1017,
          0.6493, 0.8896, 0.4739, 0.4148, 0.8231, 0.7806, 0.2423, 0.5868,
          0.5298, 0.3119, 0.1783, 0.4328, 0.7425, 0.7565, 0.5043, 0.0366,
          0.7798, 0.2880, 0.3286, 0.2891, 0.1854, 0.6603, 0.0292, 0.1024,
          0.9626, 0.4886, 0.7328, 0.0838, 0.8248, 0.9938, 0.0393, 0.2485,
          0.8241, 0.5243, 0.8722, 0.0646, 0.7489, 0.3588, 0.5249, 0.1047,
          0.8592, 0.5598]]], device='cuda:0')

In [10]:
random_layer = torch.nn.Conv1d(1,1,3)

In [11]:
print(random_layer.weight)
print(random_layer.bias)

Parameter containing:
tensor([[[-0.2464,  0.5169, -0.0164]]], requires_grad=True)
Parameter containing:
tensor([-0.1522], requires_grad=True)


In [12]:
from python_lib.saveasfile import SaveAsBin

_ = SaveAsBin(random_layer, "test", "ECAPAweights")
_.saveBoth()

In [13]:
def banana(x):
    random_layer.to("cuda")
    y = random_layer(x)
    y = torch.nn.ReLU().to("cuda")(y)
    return torch.nn.BatchNorm1d(1).to("cuda")(y)

In [14]:
y = banana(input1.to("cuda"))
print(y)
y = F.pad(y, (2,0))
y = banana(y.to("cuda"))
print(y)
y = F.pad(y, (2,0))
y = banana(y.to("cuda"))
print(y)
y = F.pad(y, (2,0))
y = banana(y.to("cuda"))
print(y)
y = F.pad(y, (2,0))
y = banana(y.to("cuda"))
print(y)

tensor([[[-0.7001,  0.3609,  0.3586, -0.7001, -0.7001,  0.8007, -0.7001,
          -0.7001, -0.7001,  3.0109, -0.7001, -0.7001, -0.7001, -0.7001,
           1.7434, -0.7001,  1.4639, -0.7001, -0.7001,  0.5053,  0.3905,
          -0.7001, -0.7001,  0.8653,  0.8230, -0.7001, -0.7001,  1.0232,
          -0.2154, -0.7001,  0.2010, -0.7001, -0.7001, -0.7001, -0.5319,
           0.5259, -0.1815, -0.7001, -0.7001,  1.8815, -0.7001, -0.7001,
          -0.7001, -0.7001,  0.8562, -0.7001, -0.7001,  2.6975, -0.7001,
           0.4409, -0.7001,  1.8822,  1.0156, -0.7001, -0.7001,  1.5201,
          -0.7001,  1.1328, -0.7001,  1.6197, -0.7001, -0.3841, -0.7001,
           2.0972]]], device='cuda:0', grad_fn=<CudnnBatchNormBackward0>)
tensor([[[-0.5141, -0.5141,  0.0305, -0.5141, -0.5141, -0.5141,  0.6934,
          -0.5141, -0.5141, -0.5141,  3.7888, -0.5141, -0.5141, -0.5141,
          -0.5141,  2.0136, -0.5141,  1.6222, -0.5141, -0.5141,  0.2313,
          -0.5141, -0.5141, -0.5141,  0.7163, -0.3

In [15]:
x2 = torch.rand([1,3,5]).to("cuda")
print(x2.shape)
print(x2)
s = x2.mean(dim=2, keepdim=True)
print(s)
print(s.shape)

torch.Size([1, 3, 5])
tensor([[[0.3755, 0.9186, 0.4761, 0.2098, 0.5713],
         [0.1765, 0.3669, 0.0833, 0.1605, 0.6393],
         [0.0220, 0.9102, 0.5144, 0.4811, 0.7107]]], device='cuda:0')
tensor([[[0.5103],
         [0.2853],
         [0.5277]]], device='cuda:0')
torch.Size([1, 3, 1])


In [16]:
(-0.3841+0.6320+0.4254-0.7384)/4

-0.016274999999999984

In [17]:
se_block1 = SEBlock(in_channels=8, 
                   se_channels=128, 
                   out_channels=8).to("cuda")

x3 = torch.rand([1,8,5]).to("cuda")
se_block1(x3).shape

torch.Size([1, 8, 5])

In [18]:
se_block1(x3)

tensor([[[0.2641, 0.4466, 0.0216, 0.3931, 0.1652],
         [0.3530, 0.3197, 0.3505, 0.3932, 0.1869],
         [0.2142, 0.0886, 0.0234, 0.3666, 0.3522],
         [0.1420, 0.3263, 0.3068, 0.2395, 0.0289],
         [0.1203, 0.2312, 0.1836, 0.2095, 0.1761],
         [0.0196, 0.4021, 0.0178, 0.4719, 0.1155],
         [0.2742, 0.3863, 0.4023, 0.1400, 0.4170],
         [0.2794, 0.0644, 0.1290, 0.3499, 0.1249]]], device='cuda:0',
       grad_fn=<MulBackward0>)

In [19]:
s = x3.mean(dim=2, keepdim=True)
s = se_block1.conv1(s)
s = se_block1.relu(s)
s = se_block1.conv2(s)
s = se_block1.sigmoid(s)

s.shape

torch.Size([1, 8, 1])

In [20]:
print(x3)
print(s)

tensor([[[0.5151, 0.8709, 0.0421, 0.7665, 0.3221],
         [0.6589, 0.5966, 0.6542, 0.7338, 0.3489],
         [0.4766, 0.1971, 0.0521, 0.8158, 0.7837],
         [0.2599, 0.5971, 0.5614, 0.4382, 0.0529],
         [0.2607, 0.5012, 0.3981, 0.4541, 0.3816],
         [0.0371, 0.7623, 0.0338, 0.8945, 0.2190],
         [0.5132, 0.7229, 0.7530, 0.2621, 0.7804],
         [0.5181, 0.1195, 0.2392, 0.6490, 0.2316]]], device='cuda:0')
tensor([[[0.5128],
         [0.5358],
         [0.4494],
         [0.5464],
         [0.4613],
         [0.5275],
         [0.5343],
         [0.5392]]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [21]:
s * x3

tensor([[[0.2641, 0.4466, 0.0216, 0.3931, 0.1652],
         [0.3530, 0.3197, 0.3505, 0.3932, 0.1869],
         [0.2142, 0.0886, 0.0234, 0.3666, 0.3522],
         [0.1420, 0.3263, 0.3068, 0.2395, 0.0289],
         [0.1203, 0.2312, 0.1836, 0.2095, 0.1761],
         [0.0196, 0.4021, 0.0178, 0.4719, 0.1155],
         [0.2742, 0.3863, 0.4023, 0.1400, 0.4170],
         [0.2794, 0.0644, 0.1290, 0.3499, 0.1249]]], device='cuda:0',
       grad_fn=<MulBackward0>)

In [22]:
a1 = torch.randn((2,1))
a2 = torch.randn((2,3))

a1*a2

tensor([[-0.2859, -0.0566,  2.4425],
        [-0.5133, -1.8580,  2.0893]])

## Dissecting ASP

Here global_context = True

lengths (in forward) = None

In [23]:
def length_to_mask(length, max_len=None, dtype=None, device=None):
    """Creates a binary mask for each sequence.

    Reference: https://discuss.pytorch.org/t/how-to-generate-variable-length-mask/23397/3

    Arguments
    ---------
    length : torch.LongTensor
        Containing the length of each sequence in the batch. Must be 1D.
    max_len : int
        Max length for the mask, also the size of the second dimension.
    dtype : torch.dtype, default: None
        The dtype of the generated mask.
    device: torch.device, default: None
        The device to put the mask variable.

    Returns
    -------
    mask : tensor
        The binary mask.

    Example
    -------
    >>> length=torch.Tensor([1,2,3])
    >>> mask=length_to_mask(length)
    >>> mask
    tensor([[1., 0., 0.],
            [1., 1., 0.],
            [1., 1., 1.]])
    """
    assert len(length.shape) == 1

    if max_len is None:
        max_len = length.max().long().item()  # using arange to generate mask
    mask = torch.arange(
        max_len, device=length.device, dtype=length.dtype
    ).expand(len(length), max_len) < length.unsqueeze(1)

    if dtype is None:
        dtype = length.dtype

    if device is None:
        device = length.device

    mask = torch.as_tensor(mask, dtype=dtype, device=device)
    return mask

In [24]:
def _compute_statistics(x, m, dim=2, eps=1e-12):
            mean = (m * x).sum(dim)
            std = torch.sqrt((m * (x - mean.unsqueeze(dim)).pow(2)).sum(dim).clamp(eps))
            return mean, std

In [104]:
# x4 -> torch.Size([10, 16, 64])

L = x4.shape[-1]

# Lengths is for batch no
# so in this case it's 10
lengths = torch.ones(x4.shape[0], device = x4.device)

mask = length_to_mask(lengths * L, max_len=L, device=x4.device)
mask = mask.unsqueeze(1)
mask.shape # 10,1,64 of all ones

# global context
total = mask.sum(dim=2, keepdim=True).float()
mean, std = _compute_statistics(x4, mask / total, eps = 1e-12)
mean = mean.unsqueeze(2).repeat(1, 1, L)
std = std.unsqueeze(2).repeat(1, 1, L)
attn = torch.cat([x4, mean, std], dim=1)

In [105]:
tdnn = TDNNBlock(16*3, 128, 1,1).to("cuda")
tanh = nn.Tanh().to("cuda")
conv = Conv1d(in_channels=128, out_channels=16, kernel_size=1).to("cuda")
attn1 = conv(tanh(tdnn(attn)))
attn1 = F.softmax(attn1, dim=2)

In [108]:
mean, std = _compute_statistics(x4, attn1)

In [111]:
mean.shape

torch.Size([10, 16])